In [801]:
import pandas as pd
import numpy as np
import os
import re
import time
import rdflib
from rdflib import Graph, Namespace, URIRef, BNode, Literal, RDF
from rdflib.namespace import NamespaceManager
from datetime import datetime

In [802]:
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
wgs84_pos = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
edm = Namespace("http://www.europeana.eu/schemas/edm/")
dc = Namespace("http://purl.org/dc/elements/1.1/")
dcterms = Namespace("http://purl.org/dc/terms/")
nvt = Namespace("http://lod.iti-germany.de/resource/")
nvto = Namespace("http://lod.iti-germany.de/schema/nvto/")
skos = Namespace("http://www.w3.org/2004/02/skos/core#")

In [803]:
class G(Graph):    
    nsm = NamespaceManager(Graph())
    nsm.bind("nvto", "http://lod.iti-germany.de/schema/nvto/")
    nsm.bind("nvt", "http://lod.iti-germany.de/resource/")
    nsm.bind("owl", "http://www.w3.org/2002/07/owl#")
    nsm.bind("rdf", "http://www.w3.org/1999/02/22-rdf-syntax-ns#")
    nsm.bind("rdfs", "http://www.w3.org/2000/01/rdf-schema#")
    nsm.bind("wgs84_pos", "http://www.w3.org/2003/01/geo/wgs84_pos#")
    nsm.bind("edm", "http://www.europeana.eu/schemas/edm/")
    nsm.bind("dc", "http://purl.org/dc/elements/1.1/")
    nsm.bind("dct", "http://purl.org/dc/terms/")

In [804]:
def bindbind(graph):
    graph.bind("nvto", "http://lod.iti-germany.de/schema/nvto/")
    graph.bind("nvt", "http://lod.iti-germany.de/resource/")
    graph.bind("owl", "http://www.w3.org/2002/07/owl#")
    graph.bind("rdf", "http://www.w3.org/1999/02/22-rdf-syntax-ns#")
    graph.bind("rdfs", "http://www.w3.org/2000/01/rdf-schema#")
    graph.bind("wgs84_pos", "http://www.w3.org/2003/01/geo/wgs84_pos#")
    graph.bind("edm", "http://www.europeana.eu/schemas/edm/")
    graph.bind("dc", "http://purl.org/dc/elements/1.1/")
    graph.bind("dcterms", "http://purl.org/dc/terms/")
    graph.bind("skos", "http://www.w3.org/2004/02/skos/core#")
    return graph

In [805]:
pr_graph = G(identifier="http://lod.iti-germany.de/contexts/productions")
ev_graph = G(identifier="http://lod.iti-germany.de/contexts/events")
vid_graph = G(identifier="http://lod.iti-germany.de/contexts/videos")
text_graph = G(identifier="http://lod.iti-germany.de/contexts/texts")
img_graph = G(identifier="http://lod.iti-germany.de/contexts/images")
aud_graph = G(identifier="http://lod.iti-germany.de/contexts/audio")
person_graph = G(identifier="http://lod.iti-germany.de/contexts/persons")
group_graph = G(identifier="http://lod.iti-germany.de/contexts/groups")
loc_graph = G(identifier="http://lod.iti-germany.de/contexts/locations")
city_graph = G(identifier="http://lod.iti-germany.de/contexts/cities")
country_graph = G(identifier="http://lod.iti-germany.de/contexts/countries")
col_graph = G(identifier="http://lod.iti-germany.de/contexts/collections")
series_graph = G(identifier="http://lGod.iti-germany.de/contexts/series")
concept_graph = G(identifier="http://lod.iti-germany.de/contexts/concepts")
graph_list = [pr_graph, ev_graph, vid_graph, text_graph, img_graph, aud_graph, person_graph, group_graph, loc_graph, city_graph, country_graph, col_graph, series_graph, concept_graph]
[n for n in pr_graph.namespaces()]
[n for n in pr_graph.nsm.namespaces()]

[('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')),
 ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')),
 ('nvto', rdflib.term.URIRef('http://lod.iti-germany.de/schema/nvto/')),
 ('nvt', rdflib.term.URIRef('http://lod.iti-germany.de/resource/')),
 ('owl', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#')),
 ('wgs84_pos', rdflib.term.URIRef('http://www.w3.org/2003/01/geo/wgs84_pos#')),
 ('edm', rdflib.term.URIRef('http://www.europeana.eu/schemas/edm/')),
 ('dc', rdflib.term.URIRef('http://purl.org/dc/elements/1.1/')),
 ('dct', rdflib.term.URIRef('http://purl.org/dc/terms/'))]

In [806]:
for i in graph_list:
    i = bindbind(i)

In [807]:
[n for n in pr_graph.namespaces()]

[('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')),
 ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')),
 ('nvto', rdflib.term.URIRef('http://lod.iti-germany.de/schema/nvto/')),
 ('nvt', rdflib.term.URIRef('http://lod.iti-germany.de/resource/')),
 ('owl', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#')),
 ('wgs84_pos', rdflib.term.URIRef('http://www.w3.org/2003/01/geo/wgs84_pos#')),
 ('edm', rdflib.term.URIRef('http://www.europeana.eu/schemas/edm/')),
 ('dc', rdflib.term.URIRef('http://purl.org/dc/elements/1.1/')),
 ('dcterms', rdflib.term.URIRef('http://purl.org/dc/terms/')),
 ('skos', rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#'))]

In [808]:
xl = pd.ExcelFile('./NVT_Metadatentabelle_MASTER.xlsm') ## load Excel File to display sheet_names

In [809]:
xl.sheet_names  ## display sheet_names

['Produktionen',
 'Ereignisse',
 'Objekte VIDEOS',
 'Objekte TEXT',
 'Objekte BILD',
 'Objekte AUDIO',
 '||_Personen',
 '||_Gruppen_Ensembles',
 '||_Veranstaltungsort',
 '||_Städte',
 '||_Länder',
 'Sammlungen',
 'Serie',
 'TYPEN und GLOSSARE',
 'Fragen und Dok',
 'TODOs',
 'Legende',
 'Unterobjekt Beispiele',
 'Übersicht']

In [810]:
def merge_columns(df, col1, col2, result=None):
    ## noch ein bisschen sauberer wäre gut, zeilenumbruch nach beschreibung vllt und personennamen in richtiger schreibweise
    if result == "season":
        col3 = "Spielzeit"
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "season_" + df["ID"] + "_" + df[col1].apply(str).str[:4] + "_" + pr_clean[col2].apply(str).str[:4]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "season_" + df["ID"] + "_" + df[col1].apply(str).str[:4]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = "season_" + df["ID"] + "_" + df[col2].apply(str).str[:4]
        return df
    elif result == "description":
        col3 = "Beschreibung_Quelle"
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "Beschreibung: " + pr_clean[col1] + " Quelle: " + pr_clean[col2]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "Beschreibung: " + pr_clean[col1]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = " Quelle: " + pr_clean[col2]
        return df
    elif result == "texts":
        col3 = "Texte_Autoren"
        df[col2] = peoplereplace(df[col2])
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "Text(e): " + pr_clean[col1] + " Autor(en): " + pr_clean[col2]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "Text(e): " + pr_clean[col1]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = "Autor(en): " + pr_clean[col2]
        return df
    else:
        print("Wrong Keyword, nothing happened")
        return df

In [811]:
def globalcolumnrename(columns):
    None
##   columns = {'Identifier / geeinigter Name':'ID', ## renames columns for future reference
##                                         'Produktionsname / Titel':'PR_Titel',
##                                         'Quelle (Beschreibung)':'Q_Beschreibung',
##                                         'Verwendete Texte':'Verwxendete_Texte',
##                                         'Autor(en) der Texte':'Autoren_Texte',
##                                         'Beteiligte Gruppen / Compagnies':'Beteiligte_Gruppen',
##                                         'Sprecher*in':'SprecherIn',
##                                         'Darsteller allgem.':'Darsteller',
##                                         'Weitere Mitwirkende':'Mitwirkende',
##                                         'Spielzeit / Laufzeit Start':'Spielzeit_Start',
##                                         'Spielzeit / Laufzeit Ende':'Spielzeit_Ende'
##                          } 
##   ## uvm, alle spaltennamen aus allen reitern holen und dann doppelte einträge entfernen 
##   ##(erst sicherstellen, dass nicht manche spaltennamen für verschiedene Bedeutungen genutzt werden)
    

In [812]:
def peoplereplace(df):
    ## temporary string beautifier for persons as literal values
    df = df.str.replace("_"," ")
    df = df.str.replace(",", " ")
    df = df.str.replace(";", "; ")
    return df

In [813]:
def refreplace(df):
    ## string converter for URIs
    df = df.str.lower()
    df = df.str.replace("ä", "ae")
    df = df.str.replace("ö","oe")
    df = df.str.replace("ü","ue")
    df = df.str.replace("ß","ss")
    df = df.str.replace("!?,\.", "")
    df = df.str.replace("[^a-zA-Z0-9;]", "_")
    df = df.str.replace("__", "_")
    df = df.str.strip("_")
    return df

In [814]:
def supersplit(df):
    ## for more convenient str splitting within the columns (expand=false), mainly for splitting all URIs
    df = df.str.split(";", expand = False) 
    return df

In [815]:
def splitreplace(df):
    ## combines refreplace and supersplit in correct order for they are both always used for any columns containing URIs
    df = df.apply(refreplace, axis=1)
    df = df.apply(supersplit, axis=1)
    return df

In [816]:
def columnref(df, sheet=None):
    ## erst Referenzliste mit Index, dann ->
    ## Referenzliste zum splitten und replacen; I=ID R=reference L=literal N=None refreplace für I und R, split für R, erst split dann refreplace
    for idx, i in enumerate(df.columns):
        idxx = str(idx).zfill(2)
        print(idxx, i)
    if sheet == "Produktionen":
        for idx, i in enumerate(df.columns):
            idxx = str(idx).zfill(2)
            if idx == 0:
                print("Referenzliste zum splitten und replacen; I=ID R=reference L=literal N=None\n\nrefreplace für I und R, split für R, erst split dann refreplace\n")
                print(["I", idxx, i])
            elif idx in range(6, 27) or idx in range(31, 35):
                print(["R", idxx, i])
            elif idx in range(2, 6):
                print(["N", idxx])
            else:
                print(["L", idxx, i])

In [817]:
def uri_list(column, predicate):
    if isinstance(row[column], list):
        for i in row[column]:
             graph.add((nvt[pr_id], predicate, nvt[str(i)]))

In [818]:
def season_list(column, column2, predicate):
    if isinstance(row[column], list):
        for i in row[column]:
             graph.add((nvt[column], predicate, Literal(row[column2])))

In [819]:
def lit_list(column, predicate):
    if isinstance(row[column], str):
        pr_graph.add((nvt[pr_id], predicate, Literal(row[column])))

In [820]:
## events = pd.read_excel('./NVT_Metadatentabelle_MASTER.xlsm', sheet_name='Ereignisse')

In [821]:
productions = pd.read_excel('./NVT_Metadatentabelle_MASTER.xlsm', sheet_name='Produktionen', header=1) ## load productions sheet with header in row 1 (row 0 is ignored i think)

In [822]:
pr_clean = (productions[productions['Identifier / geeinigter Name'].str.contains('PR', na=False)] ## drops all rows that don't start with 'PR' or are NaN
           .drop(['Musik (für GEMA)'], axis='columns') ## drops unmodeled column
           .drop([2]) ## drops example row
           .rename(columns={'Identifier / geeinigter Name':'ID', ## renames columns for future reference
                                          'Produktionsname / Titel':'PR_Titel',
                                          'Quelle (Beschreibung)':'Q_Beschreibung',
                                          'Verwendete Texte':'Verwendete_Texte',
                                          'Autor(en) der Texte':'Autoren_Texte',
                                          'Beteiligte Gruppen / Compagnies':'Beteiligte_Gruppen',
                                          'Sprecher*in':'SprecherIn',
                                          'Darsteller allgem.':'Darsteller',
                                          'Weitere Mitwirkende':'Mitwirkende',
                                          'Spielzeit / Laufzeit Start':'Spielzeit_Start',
                                          'Spielzeit / Laufzeit Ende':'Spielzeit_Ende'
                           })
            .sort_values(pr_clean.columns[0])
            .reset_index(drop=True) ## resets index to be continouus
           )

In [823]:
pr_clean = merge_columns(pr_clean, "Spielzeit_Start", "Spielzeit_Ende", result = "season")
pr_clean = merge_columns(pr_clean, "Verwendete_Texte", "Autoren_Texte", result = "texts") ## in Zukunft müssten die Autoren mit der Personentabelle abgeglichen werden, am besten trotzdem noch als literal
pr_clean = merge_columns(pr_clean, "Beschreibung", "Q_Beschreibung", result = "description")

In [824]:
pr_clean.iloc[:, [0]] = splitreplace(pr_clean.iloc[:, [0]])
pr_clean.iloc[:, 6:27] = splitreplace(pr_clean.iloc[:, 6:27])
pr_clean.iloc[:, 31:35] = splitreplace(pr_clean.iloc[:, 31:35])

In [825]:
# pr_clean.to_excel("output_pr_clean.xlsx")

In [826]:
pr_clean.loc[0,:][1]

'12 Träume des Dr. Fridkin'

In [827]:
columnref(pr_clean, sheet="Produktionen")

00 ID
01 PR_Titel
02 Beschreibung
03 Q_Beschreibung
04 Verwendete_Texte
05 Autoren_Texte
06 Produktionstyp
07 Beteiligte_Gruppen
08 Konzept
09 Textbearbeitung
10 Übersetzung
11 Regie
12 SprecherIn
13 Choreographie
14 Dramaturgie
15 Bühnenbild
16 Kostümdesgin
17 Figurenbau
18 Maskenbau
19 Lichtdesign
20 Videodesign
21 Schauspieler
22 Tänzer
23 Darsteller
24 Musiker
25 Komposition
26 Mitwirkende
27 Premierendatum
28 Dernierendatum
29 Spielzeit_Start
30 Spielzeit_Ende
31 Veranstaltungsort
32 Stadt
33 Land
34 Spielzeit
35 Texte_Autoren
36 Beschreibung_Quelle
Referenzliste zum splitten und replacen; I=ID R=reference L=literal N=None

refreplace für I und R, split für R, erst split dann refreplace

['I', '00', 'ID']
['L', '01', 'PR_Titel']
['N', '02']
['N', '03']
['N', '04']
['N', '05']
['R', '06', 'Produktionstyp']
['R', '07', 'Beteiligte_Gruppen']
['R', '08', 'Konzept']
['R', '09', 'Textbearbeitung']
['R', '10', 'Übersetzung']
['R', '11', 'Regie']
['R', '12', 'SprecherIn']
['R', '13', 'Cho

In [828]:
for index, row in pr_clean.iterrows():
    
    graph = pr_graph
    pr_id = str(row["ID"][0])
    
    pr_graph.add((nvt[pr_id], rdf.type, nvto.PerformingArtsProduction))
    
    lit_list("PR_Titel", dcterms.title)
    lit_list("PR_Titel", skos.prefLabel)
    lit_list("Premierendatum", nvto.hasFirstPerformanceDate)
    lit_list("Premierendatum", nvto.hasLastPerformanceDate)
    
    uri_list("Produktionstyp", dcterms.type)
    uri_list("Beteiligte_Gruppen", dcterms.contributor)
    uri_list("Konzept", nvto.hasConceptOriginator)
    uri_list("Textbearbeitung", nvto.hasAdaptor)
    uri_list("Übersetzung", nvto.hasTranslator)
    uri_list("Regie", nvto.hasDirector)
    uri_list("SprecherIn", nvto.hasSpeaker)
    uri_list("Choreographie", nvto.hasChoreographer)
    uri_list("Dramaturgie", nvto.hasDramaturge)
    uri_list("Bühnenbild", nvto.hasSetDesigner)
    uri_list("Kostümdesgin", nvto.hasCostumedesigner)
    uri_list("Figurenbau", nvto.hasPuppetDesigner)
    uri_list("Maskenbau", nvto.hasMaskDesigner)
    uri_list("Lichtdesign", nvto.hasLightDesigner)
    uri_list("Videodesign", nvto.hasVideoDesigner)
    uri_list("Schauspieler", nvto.hasActor)
    uri_list("Tänzer", nvto.hasDancer)
    uri_list("Darsteller", nvto.hasPerformer)
    uri_list("Musiker", nvto.hasMusician)
    uri_list("Komposition", nvto.hasComposer)
    uri_list("Mitwirkende", dcterms.contributor)
    uri_list("Veranstaltungsort", nvto.happenedAtPlace)
    uri_list("Stadt", nvto.happenedAtPlace)
    uri_list("Land", nvto.happenedAtPlace)
    uri_list("Spielzeit", nvto.hasSeason)
    season_list("Spielzeit", "Spielzeit_Start", nvto.beginsAtTime) ## ist noch buggyyyy
    season_list("Spielzeit", "Spielzeit_Ende", nvto.beginsAtTime)

In [829]:
with open('productions.ttl', 'wb') as f_ttl:
    f_ttl.write(pr_graph.serialize(format="turtle"))